# <font color='800000'>Collect Data and Construct Data Frame</font>

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime
from abrev import us_state_abbrev #made a file that has dictionary of states, with keys as whole names, and values as abreviations

### 1) Make a function that makes a data frame given a year

In [2]:
def make_dataframe(year):
    ##################### Read drug arrests by state excel files #############################
    #read the excel files labeled years
    df = pd.read_excel('drug_arrest/{}.xls'.format(year), skiprows = 3)
    
    #rename the column of interest to something concise
    df.columns; #do this to see the exact syntax for column of interest
    df = df.rename(columns={'Drug\nabuse\nviolations': 'drug_arrest'})
    df = df.rename(columns={'Drug \nabuse\nviolations': 'drug_arrest'})
    
    # only care about drug arrest numbers by state
    df = df.loc[:,['State','drug_arrest']]
    
    ############################ create state abreviation column ##############################
    url = 'https://www.archives.gov/federal-register/electoral-college/2008/popular-vote.html'
    df_elections_2009 = pd.read_html(url, header=1)[0]
    df_elections_2009 = df_elections_2009.iloc[:,:8]
    states_series = df_elections_2009.State.rename(columns={'State':'state'})
    abreviations = pd.DataFrame(states_series)[0:51]
    
    ########################## drug arrest column ############################################
    #clean up data by looking at total drug arrests
    drug_arrests = pd.Series(df['drug_arrest'])
    drug_arrests_by_state = [drug_arrests[index] for index in list(drug_arrests.index) if index%2 != 0];
    drug_arrests_series = pd.Series(drug_arrests_by_state)[0:51]
    
    ##################### year column ########################################################
    years = pd.Series([year]*52) #because
    
    ############################# include the unemployment rate column #########################
    df_unemployement = pd.read_excel('unemployement.xlsx',skiprows=5, skipfooter=2, engine= None)
    unemp = pd.Series(list(df_unemployement[year][2:])) #start at row index 2 because alabama is at that index
    
    ########################### include the sex column ####################################
    df_sex = pd.read_csv('sex/sex'+'{}'.format(year)+'.csv', skiprows=2, skipfooter=8, engine= None)
    df_sex = df_sex.drop('Location', axis=1)
    #change decimals to percentages
    df_sex *= 100
    
    ######################### include the race columns ######################################
    df_race = pd.read_csv('race/race'+'{}'.format(year)+'.csv', skiprows=2, skipfooter=14)
    df_race = df_race.drop('Location', axis=1)
    df_race = df_race.fillna(0)
    for i in range(0,3):
        column = df_race.iloc[:,i]
        for ii in range(0,len(column)):
            try:
                np.float64(column[ii])
            except ValueError:
                column[ii] = 0
    df_race = df_race.astype(dtype='float64')
    #change decimals to percentages
    df_race *= 100
    
    ######################### include the median income columns ######################################
    df_median = pd.read_excel('medianincome.xls', skiprows=59, skipfooter=1)
    df_median = pd.Series(list(df_median[year][2:]))
    #note: for 2013, it had two column values (38) and (39), we chose 39, see source file
    
    ######################### include the politics columns ######################################
    url = 'https://www.archives.gov/federal-register/electoral-college/2008/popular-vote.html'
    df_elections_2008 = pd.read_html(url, header=1)[0]
    df_elections_2008 = df_elections_2008.iloc[:,:10]

    url = 'https://www.archives.gov/federal-register/electoral-college/2012/popular-vote.html'
    df_elections_2012 = pd.read_html(url, header=1)[0]
    df_elections_2012 = df_elections_2012.iloc[:,:7]

    url = 'https://www.archives.gov/federal-register/electoral-college/2016/popular-vote.html'
    df_elections_2016 = pd.read_html(url, header=1)[0]
    df_elections_2016 = df_elections_2016.iloc[:,:8]

    #dem = 1 rep = 0
    dem2008 = list(df_elections_2008['Obama\xa0/ Biden'])
    rep2008 = list(df_elections_2008['McCain\xa0/ Palin'])
    politics2008 = pd.Series(list(map(lambda dem, rep: int(dem > rep), dem2008, rep2008)))

    dem2012 = list(df_elections_2012['Democratic Party(Obama\xa0/ Biden)'])
    rep2012 = list(df_elections_2012['Republican Party(Romney\xa0/ Ryan)'])
    politics2012 = pd.Series(list(map(lambda dem, rep: int(dem > rep), dem2012, rep2012)))

    dem2016 = list(df_elections_2016['Democratic Party(Clinton\xa0/ Kaine)'])
    rep2016 = list(df_elections_2016['Republican Party(Trump\xa0/ Pence)'])
    politics2016 = pd.Series(list(map(lambda dem, rep: int(dem > rep), dem2016, rep2016)))
    
    # political views will change with each election
    if year <= 2012:
        # put the dataframe together with proper poltical view
        df_all = pd.concat([years, abreviations, drug_arrests_series, df_race,\
                        df_median, df_sex, unemp, politics2008], axis = 1)
    elif year > 2012 and year <= 2016:
        # put the dataframe together with proper poltical view
        df_all = pd.concat([years, abreviations, drug_arrests_series, df_race,\
                        df_median, df_sex, unemp, politics2012], axis = 1)
    else:
        # put the dataframe together with proper poltical view
        df_all = pd.concat([years, abreviations, drug_arrests_series, df_race,\
                        df_median, df_sex, unemp, politics2016 ], axis = 1)
    
    #label each column properly
    df_all.columns = ['year','state','drug_arrests', 'white', 'black',\
                             'hispanic', 'median_income','male_percentage',\
                             "unemployment_rate","politics_dem0_rep1"] 
    df_all = df_all.drop(51)
    return df_all


### 2) Construct data frame for years 2014-2018 using the method defined above

In [3]:
combined = []
for year in range(2014,2018):
    combined.append(make_dataframe(year))
combinedd = pd.concat(combined, axis = 0, ignore_index=True)  


/Users/PB/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/Users/PB/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/Users/PB/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### 3) Add the Dependent Variable : mental health column

In [4]:
# read the population data. needed to multiple the 2017 mental health data
# because 2017 is the only one given in percentages
df_population_all = pd.read_csv('mental_health/nst-est2017-alldata.csv')
population_data = df_population_all.iloc[5:56, 4:15]
pop2017 = pd.Series(list(population_data.POPESTIMATE2017), name='pop')

In [5]:
##################### Mental Health 2017
xls2017 = pd.ExcelFile('mental_health/NSDUHsaeExcelTabs2017.xlsx')
# rmhs = Received Mental Health Services
df_rmhs = pd.read_excel(xls2017, 29, skiprows=10)
df_rmhs = df_rmhs.iloc[:,1:3]

# smi = Serious Mental Illness
df_smi = pd.read_excel(xls2017, 27, skiprows=10)
df_smi = df_smi.iloc[:,1:3]

# put both mental health columns together
mental_2017 = pd.merge(df_rmhs, df_smi, on = ['West'], how = 'outer')
mental_2017.columns = ['state', 'rmhs_p', 'smi_p']
mental_2017 = mental_2017.replace({'state':us_state_abbrev})
mental_2017.insert(loc=0, column='year', value=2017)
mental_2017.insert(loc=4, column='pop', value=pop2017)

# need to multiply population for 2017 with percentage to match units for other years
rmhs = round(mental_2017['rmhs_p'] * mental_2017['pop'] / 1000,0)
smi = round(mental_2017['smi_p'] * mental_2017['pop'] / 1000, 0)

mental_2017.insert(loc=2,value=rmhs, column='rmhs')
mental_2017.insert(loc=3,value=smi, column='smi')
mental_2017 = mental_2017.drop(labels=['smi_p', 'rmhs_p', 'pop'], axis=1)

In [6]:
##################### Mental Health 2016
# read file for 2016
xls2016 = pd.ExcelFile('mental_health/NSDUHsaeTotals2016.xlsx')

#make rmhs column
df_rmhs = pd.read_excel(xls2016, 28, skiprows=11)
df_rmhs = df_rmhs.iloc[:,1:3]

#make smi column
df_smi = pd.read_excel(xls2016, 26, skiprows=11)
df_smi = df_smi.iloc[:,1:3]

#merge the two columns together
mental_2016 = pd.merge(df_rmhs, df_smi, on = ['West'], how = 'outer')
mental_2016.columns = ['state', 'rmhs', 'smi']
mental_2016 = mental_2016.replace({'state':us_state_abbrev})
mental_2016.insert(loc=0, column='year', value=2016)

In [7]:
##################### Mental Health 2015
# read file for 2015
xls2015 = pd.ExcelFile('mental_health/NSDUHsaeTotals2015.xlsx')

# make smi column (note: there is no rhms data for this year)
df_smi = pd.read_excel(xls2015, 12, skiprows=11)
mental_2015 = df_smi.iloc[:,1:3]
mental_2015.columns = ['state', 'smi']
mental_2015 = mental_2015.replace({'state':us_state_abbrev})
mental_2015.insert(loc=0, column='year', value=2015)

In [8]:
##################### Mental Health 2014
#read file for 2014
xls2014 = pd.ExcelFile('mental_health/NSDUHsaeTotals2014.xlsx')

# make smi column (note: there is no rhms data for this year)
df_smi = pd.read_excel(xls2014, 23, skiprows=10)
mental_2014 = df_smi.iloc[:,1:3]
mental_2014.columns = ['state', 'smi']
mental_2014 = mental_2014.replace({'state':us_state_abbrev})
mental_2014.insert(loc=0, column='year', value=2014)

In [9]:
# merge all the mental health dataframes together
df_iter1 = pd.concat([mental_2014, mental_2015, mental_2016, mental_2017],\
                   axis=0, ignore_index=True, sort=False)


### 4) Add another independent variable : lab counts column

In [10]:
######################## scrape from DEA Lab location site #####################
#specify date range for the url
date1 = '2014-01-01'
date2 = '2017-12-31'
dates = pd.date_range(date1, date2).tolist()
lab_locations = []

for date in dates:
    day = date.date().day
    month = date.date().month
    year = date.date().year
    
    #scrape DEA site
    url = requests.get('https://www.dea.gov/clan-lab?state=All&date={}%2F{}%2F{}'.format(month,day,year))
    r = url.content
    soup = BeautifulSoup(r,'html.parser')
    states = soup.findAll('td',{'headers':'view-field-clan-lab-address-administrative-area-table-column'},
            {'class':'views-field views-field-field-clan-lab-address-administrative-area is-active'})
    
    #needed to strip off spaces and unnecessary descriptors off states
    states_for_date = [(state.text).strip('\nState:').strip() for state in states]
    years = [year]*(len(states_for_date))
    combo = list(zip(states_for_date,years))
    lab_locations += combo

In [11]:
#this is a list of states
states = sorted(list(set([lab[0] for lab in lab_locations])))[1:]

#count the number of labs in a state for a certain year
lab_counts = [(year, state, lab_locations.count((state,year))) for year in range(2014,2018) for state in states]

#make the other dataframe that includes lab counts
table = []
for tup in lab_counts:
    #tup0 = year, tup1 = abrev, tup2 = lab count
    table.append((tup[0], us_state_abbrev[tup[1]], tup[2]))

lab_df = pd.DataFrame(table, columns = ['year', 'state','lab_count'])

### 5) Put all data frames together

In [12]:
# merge the first data frame with the other independent variable, lab_count
df_iter2 = pd.merge(combinedd, lab_df, on = ['state','year'], how = 'outer')

#merge the dataframe above with the mental health dataframe
full_df = pd.merge(df_iter2,df_iter1, on = ['state','year'], how = 'outer')

full_df.isna().sum(); #check where N/A values are. neglect rhms column N/A values

full_df['lab_count'] = full_df.lab_count.fillna(0)
full_df = full_df[['year', 'state', 'rmhs', 'smi','drug_arrests','white', 'black', 'hispanic', 'median_income', 'male_percentage','unemployment_rate', 'lab_count','politics_dem0_rep1']]

In [1]:
full_df.head(), full_df.tail

NameError: name 'full_df' is not defined

In [14]:
full_df.to_pickle("dataframe.pkl")